# dataset mergeing and cleaning

In [2]:
import numpy as np
import random
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import os
import requests

In [ ]:
admission =     pd.read_csv(r"F:\SchoolWork\research\Med-BERT\datasets\admissions.csv")
diagnoses_icd = pd.read_csv(r"F:\SchoolWork\research\Med-BERT/datasets/diagnoses_icd.csv")
drgcodes =      pd.read_csv(r"F:\SchoolWork/research/Med-BERT\datasets\drgcodes.csv")

# Part 1, Data preparation
## Step 1, map ICD-9 into ICD-10;
truncate all diagnose code into 3 digits

In [ ]:
#mapping ICD-9 TO ICD-10
gem_df = pd.read_csv(r"F:\SchoolWork\research\Med-BERT\datasets\icd9toicd10cmgem.csv")
# 显示文件前几行以了解数据结构
print(gem_df.head())

# 创建一个字典用于ICD-9到ICD-10的映射
icd9_to_icd10 = dict(zip(gem_df['icd9cm'], gem_df['icd10cm']))

# 测试映射
icd9_code = '0010'  # 替换为你要映射的ICD-9代码
icd10_code = icd9_to_icd10.get(icd9_code, 'MAPPING UNFOUND')

print(f'ICD-9代码 {icd9_code} 对应的ICD-10代码是: {icd10_code}')

In [ ]:
diagnoses_icd.loc[diagnoses_icd['icd_version'] == 9, 'icd_code'] = diagnoses_icd.loc[diagnoses_icd['icd_version'] == 9, 'icd_code'].apply(lambda x: icd9_to_icd10.get(x, 'MAPPING UNFOUND'))
diagnoses_icd['icd_code'] = diagnoses_icd['icd_code'].str.slice(0, 3)
diagnoses_icd.head(30)

In [ ]:
drgcodes.columns
diagnoses_icd.columns
#admission.columns

## step 2, construct a file with index of hadm_list 

In [ ]:
hadm_list = pd.unique(admission['hadm_id'])

data_diag is an array with columns of 'patient_sk','admit_dt_tm','discharge_dt_tm', 'third_party_ind','diagnosis',
admission中的subject_id为patient_sk
admission中的admittime为admit_dt_tm
admission中的dischtime为discharge_dt_tm
admission中的insurance为third_party_ind
diagnoses_icd中的icd_code为diagnosis
将admission['hadm_id']作为匹配索引，每行都有一个单独的diagnosis

In [15]:
admission.rename(columns={
    'subject_id': 'patient_sk',
    'admittime': 'admit_dt_tm',
    'dischtime': 'discharge_dt_tm',
    'insurance': 'third_party_ind'
}, inplace=True)

diagnoses_icd.rename(columns={
    'icd_code': 'diagnosis'
}, inplace=True)

# 将admission和diagnoses_icd数据集进行合并，按hadm_id进行匹配
merged_data = pd.merge(admission, diagnoses_icd, on='hadm_id', how='left')

# 保留需要的列
final_data = merged_data[['patient_sk', 'admit_dt_tm', 'discharge_dt_tm', 'third_party_ind', 'diagnosis']]


NameError: name 'admission' is not defined

 Convert time into ymd form

In [ ]:
final_data['admit_dt_tm'] = final_data['admit_dt_tm'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d"))


In [ ]:
final_data['discharge_dt_tm'] = final_data['discharge_dt_tm'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d"))

In [ ]:

# 保存到新的CSV文件
final_data.to_csv('diagdatashort.csv', index=False)

# 打印前几行以检查结果
print(final_data.head())

# PART 2, DATA PREPROCESSING

In [16]:
pd.read_csv(r"F:/SchoolWork/research/Med-BERT/Pretraining Code/Data Pre-processing Code\diagdatashort.csv").columns

Index(['patient_sk', 'admit_dt_tm', 'discharge_dt_tm', 'third_party_ind',
       'diagnosis'],
      dtype='object')

In [2]:
import sys
sys.path.append("F:/SchoolWork/research/Med-BERT/Pretraining Code/Data Pre-processing Code")
from gpt_data_processing_code import main


# 设置参数
diagFile = r"F:\SchoolWork\research\Med-BERT\Pretraining Code\Data Pre-processing Code\diagdatashort.csv"
typeFile = "NA"  # 如果没有类型文件，则设为 None
outFile = "dataprocessed"
p_samplesize = 1000  # 设置采样大小，如果不需要采样则设为 0

# 调用 main 函数
main(diagFile, typeFile, outFile, p_samplesize)

Data file


c:\Users\xitia\anaconda3\envs\medbert_env\lib\site-packages\ipykernel_launcher.py:13: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  del sys.path[0]


Sampling
Start Data Preprocessing !!!
Processed 1000 pts
Splitting


# PART 2.5, transforming the test, valid, train data into TFRecords

In [3]:
!pip3 install pickle5

In [1]:
import sys
sys.path.append("F:/SchoolWork/research/Med-BERT/Pretraining Code/Data Pre-processing Code")
import gpt_data_prep as prep
input_file0 = "F:/SchoolWork/research/Med-BERT/Pretraining Code/Data Pre-processing Code/dataprocessed.bencs.train"
output_file = 'F:/SchoolWork/research\Med-BERT/Pretraining Code/Data Pre-processing Code/prepared/outputtrain.tfrecord'
vocab_file = 'F:/SchoolWork/research/Med-BERT/Pretraining Code/Data Pre-processing Code/dataprocessed.types'
prep.preprocess_data(input_file0, output_file, vocab_file, max_seq_length=32, max_predictions_per_seq=2, masked_lm_prob=0.2, random_seed=12345)


c:\Users\xitia\anaconda3\envs\medbert_env\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\xitia\anaconda3\envs\medbert_env\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\xitia\anaconda3\envs\medbert_env\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\xitia\anaconda3\envs\medbert_env\lib\site-pack

2
2
INFO:tensorflow:*** Example ***
INFO:tensorflow:input_ids: 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:masked_lm_positions: 0 0 0
INFO:tensorflow:masked_lm_ids: 3 0 0
INFO:tensorflow:masked_lm_weights: 1.0 0.0 0.0
INFO:tensorflow:next_sentence_labels: 1
1
INFO:tensorflow:*** Example ***
INFO:tensorflow:input_ids: 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:masked_lm_positions: 0 0 0
INFO:tensorflow:masked_lm_ids: 3 0 0
INFO:tensorflow:masked_lm_weights: 1.0 0.0 0.0
INFO:tensorflow:next_sentence_labels: 0
INFO:tensorflow:*** Example ***
INFO:tensorf

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\xitia\anaconda3\envs\medbert_env\lib\logging\__init__.py", line 1025, in emit
    msg = self.format(record)
  File "c:\Users\xitia\anaconda3\envs\medbert_env\lib\logging\__init__.py", line 869, in format
    return fmt.format(record)
  File "c:\Users\xitia\anaconda3\envs\medbert_env\lib\logging\__init__.py", line 608, in format
    record.message = record.getMessage()
  File "c:\Users\xitia\anaconda3\envs\medbert_env\lib\logging\__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "c:\Users\xitia\anaconda3\envs\medbert_env\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\Users\xitia\anaconda3\envs\medbert_env\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\Users\xitia\anaconda3\envs\medbert_env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
 

INFO:tensorflow:Wrote 2099 total instances


# Part 3, running the pre-training model

## Step 1, running the pretraining Med-Bert for Pretraining

In [2]:
import sys
sys.path.append("F:/SchoolWork/research/Med-BERT/Pretraining Code")
#from gptQA2Seq import main_function   #where we make sure that LOS classification task is using the full sequence rather than the first token

In [3]:
input_file = r"F:/SchoolWork/research/Med-BERT/Pretraining Code/Data Pre-processing Code/dataprocessed.bencs.train,F:/SchoolWork/research/Med-BERT/Pretraining Code/Data Pre-processing Code/dataprocessed.bencs.valid, F:/SchoolWork/research/Med-BERT/Pretraining Code/Data Pre-processing Code/dataprocessed.bencs.test"
bert_config_file = r"F:/SchoolWork/research/Med-BERT/Pretraining Code/config.json"
output_dir = r"F:/SchoolWork/research/Med-BERT/Pretraining Code/BertQA2Seq"

#main_function(input_files, bert_config_file, output_dir)



In [2]:
import sys
import tensorflow as tf
import os
sys.path.append("F:/SchoolWork/research/Med-BERT/Pretraining Code")
import gptpretraining
input_file = "F:/SchoolWork/research\Med-BERT/Pretraining Code/Data Pre-processing Code/prepared/outputtrain.tfrecord"
bert_config_file = os.path.join("F:/SchoolWork/research/Med-BERT/Pretraining Code/config.json")
output_dir = os.path.join("F:/SchoolWork/research/Med-BERT/Pretraining Code/Bertmodel")
gptpretraining.main(bert_config_file, input_file, output_dir)

INFO:tensorflow:*** Input Files ***
INFO:tensorflow:  F:\SchoolWork\research\Med-BERT\Pretraining Code\Data Pre-processing Code\prepared\outputtrain.tfrecord

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': 'F:/SchoolWork/research/Med-BERT/Pretraining Code/Bertmodel', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spe

InvalidArgumentError: Key: masked_lm_positions.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseSingleExample}}]]
	 [[node IteratorGetNext (defined at c:\Users\xitia\anaconda3\envs\medbert_env\lib\site-packages\tensorflow_estimator\python\estimator\util.py:110) ]]

In [25]:
input_file = "F:/SchoolWork/research/Med-BERT/Pretraining Code/Data Pre-processing Code/dataprocessed.bencs.train,F:/SchoolWork/research/Med-BERT/Pretraining Code/Data Pre-processing Code/dataprocessed.bencs.valid,F:/SchoolWork/research/Med-BERT/Pretraining Code/Data Pre-processing Code/dataprocessed.bencs.test"
input_files=[]
for input_pattern in input_file.split(","):
    input_files.extend(tf.gfile.Glob(input_pattern))
print(input_files)

import tensorflow as tf

def is_tfrecord(filenames):
  for filename in filenames:
    try:
        record_iterator = tf.compat.v1.io.tf_record_iterator(path=filename)
        for string_record in record_iterator:
            return True
    except tf.errors.DataLossError:
        return False

# Replace 'your_file.tfrecord' with the path to your file
file_path = input_files
if is_tfrecord(file_path):
    print(f"{file_path} is a TFRecord file.")
else:
    print(f"{file_path} is not a TFRecord file.")

['F:\\SchoolWork\\research\\Med-BERT\\Pretraining Code\\Data Pre-processing Code\\dataprocessed.bencs.train', 'F:\\SchoolWork\\research\\Med-BERT\\Pretraining Code\\Data Pre-processing Code\\dataprocessed.bencs.valid', 'F:\\SchoolWork\\research\\Med-BERT\\Pretraining Code\\Data Pre-processing Code\\dataprocessed.bencs.test']
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
['F:\\SchoolWork\\research\\Med-BERT\\Pretraining Code\\Data Pre-processing Code\\dataprocessed.bencs.train', 'F:\\SchoolWork\\research\\Med-BERT\\Pretraining Code\\Data Pre-processing Code\\dataprocessed.bencs.valid', 'F:\\SchoolWork\\research\\Med-BERT\\Pretraining Code\\Data Pre-processing Code\\dataprocessed.bencs.test'] is not a TFRecord file.
